<a href="https://colab.research.google.com/github/JesseDiGiacomo/WebScraping/blob/main/XCBrasilWebScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The goal of this job was to collect from the National Database Site (XCBrasil.com.br) all the official flights from the city of Petropolis / RJ and analyze them. 

**The aim was to answer:**


*   *Who is the most frequent pilot of all times*
*   *The longest flight of the city*
*   *The fartest flight of the city*
*   *The longest flight of each take-off*
*   *The fartest flight of each take-off*
*   *More frequent take-off of all times*
*   *More frequent months of each take-off*

To start the web scraping we analyzed the XCBrasil.com.br and discovered we could filter the flights by take-off location.
The city of Petropolis has three (3) take-offs. They are logged into fifteen (15) different take-offs due to misspelling errors. 

We picked one of the registered take-offs to test the best way to proceed.

First, conect to Google Drive so we can store the data acquired.

# Imports and Conections

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Them stablish where we want to store it.



In [143]:
path = "/content/drive/MyDrive/XCBrasil_flights/"

Now we import the libraries to work with

In [144]:
# Imports
import os
import csv
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import statsmodels
import statsmodels.api as sm
import matplotlib.patches as mpatches
from statsmodels.formula.api import ols
from tabulate import tabulate
from pandas_profiling import ProfileReport

%matplotlib inline

Then we create the CSV file with the header and open it.

In [163]:
# Creates the Final Data file
outputfile = open((path + 'FlightsData.csv'), 'a', encoding='utf_8')
writer = csv.writer(outputfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_ALL)

# Adds Header
writer.writerow(['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points'])

80

#Testing

Stabilish the URL and start scraping and testing

In [23]:
# XCBrasil homepage
base_url = 'http://xcbrasil.com.br'
xcbrasil_url = 'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892'

In [24]:
lst = []

condition = True

while condition: 
  # Use requests to retrieve data from a given URL
  xcbrasil_response = requests.get(xcbrasil_url)

  # Parse the whole HTML page using BeautifulSoup
  xcbrasil_soup = BeautifulSoup(xcbrasil_response.text, 'html.parser')

  # Title of the parsed page
  xcbrasil_soup.title

  # Reading table
  flights = xcbrasil_soup.find('table', {'class':'listTable'})

  for row in flights.find_all('tr')[1:-1]:
      s = pd.Series([data.text for data in row.find_all('td')])
      lst.append(s)
  
  # Next Page
  pg = xcbrasil_soup.find('div', {'class':'pagesDiv'})
  active_pg = pg.find('div', {'class':'activePageNum'})
  
  # Test if is Blank
  if active_pg is None:
    condition = False
  if active_pg.findNextSibling('a') is None:
    condition = False
  else:
    next_url = base_url + active_pg.findNextSibling('a').get("href")
    xcbrasil_url = next_url

In [25]:
# Concatenate the Pandas Series in a DataFrame
data = pd.concat(lst, axis=1).T

In [26]:
data.tail()

,0,1,2,3,4,5,6,7,8,9
3043,3106,17/01/2006,Daniel Wenna\nPetropolis City (Simeria takeof...,0:33,4.7 km,6.7 km,10.08,,,
3044,3107,15/09/2001,Artur Pereira\nPetropolis City (Simeria take...,1:43,5.2 km,18.5 km,27.76,,,
3045,3108,09/06/2001,Artur Pereira\nPetropolis City (Simeria take...,1:47,7.0 km,19.4 km,29.04,,,
3046,3109,06/05/2001,Artur Pereira\nPetropolis City (Simeria take...,1:48,25.3 km,32.3 km,48.39,,,
3047,3110,13/05/2000,Artur Pereira\nPetropolis City (Simeria take...,1:13,13.5 km,18.8 km,28.23,,,


We need to delete some columns we don't need and split the ones with information regarding the pilot and take-off site. Then we create the CSV file with the header and open it.

In [27]:
data = data.drop(columns=[0,7,8,9])
data.head()

,1,2,3,4,5,6
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34


In [28]:
data.columns = ['Date', 'Pilot and Takeoff', 'Duration', 'Straight Distance', 'OLC KM', 'OLC Points']

In [29]:
data.head()

,Date,Pilot and Takeoff,Duration,Straight Distance,OLC KM,OLC Points
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34


In [30]:
df2 = data['Pilot and Takeoff'].str.split('\n', expand=True)
df2.drop(columns=[2], inplace=True)
df2.columns = ['Pilot', 'Location']
df2.head()

,Pilot,Location
0,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [31]:
data = data.join(df2)

In [32]:
data.head()

,Date,Pilot and Takeoff,Duration,Straight Distance,OLC KM,OLC Points,Pilot,Location
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [33]:
data = data.drop(columns=['Pilot and Takeoff'])
data.head()

,Date,Duration,Straight Distance,OLC KM,OLC Points,Pilot,Location
0,07/06/2022,0:39,5.4 km,8.0 km,12.04,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,07/06/2022,0:30,4.8 km,7.1 km,10.70,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,07/06/2022,0:13,4.6 km,5.4 km,8.05,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,05/06/2022,0:48,6.1 km,9.7 km,14.62,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,22/05/2022,0:48,4.7 km,11.6 km,17.34,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [34]:
data = data[['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points']]
data.head()

,Date,Pilot,Location,Duration,Straight Distance,OLC KM,OLC Points
0,07/06/2022,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6],0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6],0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia,Petropolis City (Simeria takeoff) - BR,0:48,4.7 km,11.6 km,17.34


Now we build a routine to scrap all the fifteen takeoffs registered for Petropolis/RJ, save them to a CSV file, and close it.

# Web Scraping Data

In [164]:
#Scrapping others take offs
base_url = 'http://xcbrasil.com.br'
takeoff_url_list = [
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9222',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9236',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9276',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8897',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8960',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8900',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9333',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:13051',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9495',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9283',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9275',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8945',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:11051',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892'
                    ]
                
for takeoff in takeoff_url_list:
  xcbrasil_url = takeoff

  lst = []

  condition = True

  while condition: 
    # Use requests to retrieve data from a given URL
    xcbrasil_response = requests.get(xcbrasil_url)

    # Parse the whole HTML page using BeautifulSoup
    xcbrasil_soup = BeautifulSoup(xcbrasil_response.text, 'html.parser')

    # Title of the parsed page
    xcbrasil_soup.title

    # Reading table
    flights = xcbrasil_soup.find('table', {'class':'listTable'})

    for row in flights.find_all('tr')[1:-1]:
        s = pd.Series([data.text for data in row.find_all('td')])
        lst.append(s)
    
    # Next Page
    pg = xcbrasil_soup.find('div', {'class':'pagesDiv'})
    active_pg = pg.find('div', {'class':'activePageNum'})
    
    
    # Test if is Blank
    if active_pg is None:
      condition = False
    else:
      if active_pg.findNextSibling('a') is None:
        condition = False
      else:
        next_url = base_url + active_pg.findNextSibling('a').get("href")
        xcbrasil_url = next_url

  # Concatenate the Pandas Series in a DataFrame
  data = pd.concat(lst, axis=1).T

  # Dropping non usable columns
  data = data.drop(columns=[0,7,8,9])

  # Naming Columns
  data.columns = ['Date', 'Pilot and Takeoff', 'Duration', 'Straight Distance', 'OLC KM', 'OLC Points']

  # Spliting Pilot and Takeoff
  df2 = data['Pilot and Takeoff'].str.split('\n', expand=True)
  df2.drop(columns=[2], inplace=True)
  df2.columns = ['Pilot', 'Location']
  df2.head()
  data = data.join(df2)
  data = data.drop(columns=['Pilot and Takeoff'])

  # Reorganizing Columns
  data = data[['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points']]
  
  # Cleaning Values
  data['Straight Distance'] = data['Straight Distance'].str.replace('km','')
  data['OLC KM'] = data['OLC KM'].str.replace('km','')
  data = data[data.iloc[:, 0] != data.columns[0]]

  # Writing to CSV
  data.to_csv(outputfile, index=False, header=False)

print('Scraping Done')

Scraping Done


In [165]:
outputfile.close()

# Analysing Data

In [168]:
df = pd.read_csv((path + 'FlightsData.csv'), encoding='utf_8')
df = pd.DataFrame(df)
df.describe()

,Date,Pilot,Location,Duration,Straight Distance,OLC KM,OLC Points
count,10645,10645,10645,10645,10645,10645,10645
unique,2253,525,103,257,610,741,3845
top,21/04/2020,Marcio Montenegro,Petrópolis- Parque São Vicente - BR,0:12,4.5,5.5,8.26
freq,42,670,3953,267,645,157,24


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10645 entries, 0 to 10644
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               10645 non-null  object
 1   Pilot              10645 non-null  object
 2   Location           10645 non-null  object
 3   Duration           10645 non-null  object
 4   Straight Distance  10645 non-null  object
 5   OLC KM             10645 non-null  object
 6   OLC Points         10645 non-null  object
dtypes: object(7)
memory usage: 582.3+ KB


# Most frequent Pilot of all times

In [170]:
# Most frequent Pilot of all times
df['Pilot'].mode()

0     Marcio Montenegro
dtype: object

In [207]:
df['Pilot'].value_counts()

 Marcio Montenegro                     670
 luiz fayao                            435
 ricardo campos arnaud de veiga fl     404
 Luan lima bastos                      335
 Ronaldo Kiyoshi Nakayama              316
                                      ... 
 Ivan Marangon Schwanes                  1
 Augusto Glauco de Almeida Quintino      1
 Douglas  Goncalves                      1
 Rui Miguel Mendes e Menezes Santos      1
 marcelo tabajara ferraz d veiga         1
Name: Pilot, Length: 525, dtype: int64

# More frequent Take off of all times ***UNIFICAR RAMPAS***

In [171]:
# More frequent Take off of all times
df['Location'].mode()

0    Petrópolis- Parque São Vicente - BR
dtype: object

In [204]:
df['Location'].value_counts()

Petrópolis- Parque São Vicente - BR              3953
Petropolis City (Simeria takeoff) - BR           2636
Petropolis City (Simeria takeoff) - BR [~0.6]    2532
Petropolis City (Simeria takeoff) - BR [~0.5]     734
Petropolis - BR                                   123
                                                 ... 
Rampa Simeria Petrópolis - BR [~2.8]                1
Rampa Simeria Petrópolis - BR [~1.4]                1
Petropolis - Alto Morin - BR [~5.0]                 1
Petropolis - Alto Morin - BR [~2.3]                 1
Petropolis - Alto Morin - BR [~4.5]                 1
Name: Location, Length: 103, dtype: int64

# Longest flight

In [172]:
# Longest flight
df['Duration'].max()

'7:05'

# Fartest flight

In [183]:
# Fartest flight
df['Straight Distance'] = df['Straight Distance'].astype(np.float64)
np.float64(df['Straight Distance']).max()

156.3